In [8]:
from os import walk
from joblib import Parallel, delayed
import librosa
import numpy as np
import multiprocessing
import json
from deep_audio import Directory, Audio, JSON

In [9]:
num_cores = multiprocessing.cpu_count()

sampling_rate = 22050

path = f'audios/{sampling_rate}'

f = Directory.filenames(path)

data = {
    "mapping": [],
    "lpcc": [],
    "labels": []
}

In [10]:
def process_directory(dir, index):
    signal, sr = Audio.read(f'{path}/{dir}', sr=sampling_rate, normalize=False)

    signal = np.array(signal)

    signal = signal[:len(signal) - len(signal) % (sr * 5)]

    segments = len(signal) // (sr * 5)

    m = {
        "lpc": [],
        "labels": [index] * segments
    }

    for i in range(segments):
        start_sample = sr * i * 5
        finish_sample = start_sample + (sr * 5)

        sample = signal[start_sample:finish_sample]

        lpcc = Audio.lpcc(sample, rate=sr)

        lpcc = lpcc.T

        m['lpcc'].append(lpcc.tolist())

    print(f'{dir} -> segments: {segments}')
    return m

In [11]:
def object_lpcc_to_json(m):
    data['mapping'] = [file.replace('.wav', '') for file in f]

    for i in m:
        data['lpcc'].extend(i['lpcc'])
        data['labels'].extend(i['labels'])

    JSON.create_json_file(f'processed/lpcc/lpcc_{sampling_rate}.json', data)


In [12]:
if __name__ == '__main__':
    m = Parallel(n_jobs=num_cores, verbose=len(f), temp_folder='./tmp/')(
        delayed(process_directory)(i, j) for j, i in enumerate(f) if i is not None)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   36.6s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   48.8s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   51.6s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   54.7s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   54.9s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:   57.9s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  2.0min
[Parallel(

In [13]:
    object_lpcc_to_json(m)

In [14]:
data['labels']

17712